## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('Base_lattice_2.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

## Env Lattices

In [3]:
Bv = np.random.randint(1, 20, (np.shape(avail_lattice)), dtype='l')
Gv = np.random.randint(1, 20, (np.shape(avail_lattice)), dtype='l')
Yv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Rv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Vv = np.random.randint(1, 99, (np.shape(avail_lattice)), dtype='l')

In [4]:
Vv

array([[[70, 87, 63, 13, 67, 60,  1, 25, 94, 93, 97],
        [ 5, 73, 19, 46, 93, 72, 73, 17, 87, 80, 21],
        [49, 86, 88, 98, 67, 49, 91, 12, 25, 16, 26],
        [ 6, 23, 55, 52, 35, 30, 13, 45, 59, 21, 64],
        [47, 98, 76, 47, 95, 12, 71, 96, 35, 42, 22],
        [75, 92, 82, 58,  7, 12, 62, 94, 78, 20,  8],
        [58, 17, 71, 41, 80, 63, 67, 11, 42, 43, 41],
        [30, 75, 75, 32, 82, 37, 62, 82, 86, 28, 26],
        [48, 55, 73, 98, 74, 12, 49, 93, 17, 67, 73],
        [34, 49, 15, 41, 66, 47, 96, 23, 65, 77, 41],
        [65, 93, 32, 61, 29, 58, 84, 97, 44, 69, 82],
        [94, 27, 90, 83, 52, 85, 48, 58, 42, 72, 29],
        [33, 13, 89, 72, 74, 61, 40, 46, 34, 95, 61],
        [35, 67, 30, 40, 64, 23, 54, 65, 27, 69, 44],
        [48, 10, 71, 30,  6,  2, 87, 27, 94,  6, 17],
        [27, 45, 27, 79, 89,  9, 20, 94, 60, 76, 47],
        [22, 67, 62, 58, 27, 54, 69, 23, 83, 39, 66],
        [ 5, 20, 92,  8, 94, 89, 58, 77, 70, 45, 14],
        [39, 80,  1, 52, 69,

## Stencils

In [5]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [6]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [7]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [8]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

## Deriving all possible agent Center points

In [9]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
B1 = int(mass_size / 5)
G1= int( mass_size / 5)
Y1=int( mass_size / 7.5)
R1= int(mass_size / 7.5)
V1= int(mass_size -(B1+G1+Y1+R1))
Num_of_voxels_lst = np.array([B1,G1,Y1,R1,V1])
Cube_root_list = Num_of_voxels_lst ** (1/3)
Radius_list = Cube_root_list.astype(int) 
Maximum_radius = int(Radius_list[np.argmax(Radius_list)])

Num_of_voxels_lst

array([136, 136,  91,  91, 230])

In [10]:
#Divide the available lattice into grids to generate the necessary locations for origins??

In [11]:
avail_lattice_copy = np.copy(avail_lattice)
avail_lattice_copy[2].shape

(21, 11)

In [12]:
#slices= np.arange(1,20,3)
x_cordinate= np.arange(3,21,7,dtype=int)
y_cordinate = np.array([2,8],dtype=int)
indexes= []
for item in x_cordinate :
    for num in y_cordinate:
        a=item,num
        indexes.append(a)

In [13]:
indexes

[(3, 2), (3, 8), (10, 2), (10, 8), (17, 2), (17, 8)]

In [14]:
for num in indexes:
    selected_slice= avail_lattice_copy[2]
    selected_slice[num[0]][num[1]]=2
index_lattice_flat = avail_lattice_copy.flatten()    
indexing =list(np.where(index_lattice_flat ==2))
All_coordinates=[element for tupl in indexing for element in tupl]

In [15]:
All_coordinates

[497, 503, 574, 580, 651, 657]

In [16]:
# All possible locations isolate the 2d array and find the points 
locations= [194,190,257,253,307,311]

## Permutation Combinations for further simulation verifications

In [17]:
all_permutations = (list(itertools.permutations(All_coordinates, 5)))
all_permutations[618]

(657, 497, 651, 503, 574)

In [18]:
#all_permutations.index((574,503,651,497,580))
all_permutations.index((657,574,580,651,497))

664

## Agent Class

In [19]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = neighs_availibility_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(self.origin)
        
        

## Initilize Agents

In [20]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

## Enviornment Class

In [21]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

# Blue function max simulation

In [22]:
env_availability_viz_blue= []
env_availability_score_blue= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_B = {"availibility": occ_lattice_sim,"enviornment": Bv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_B, Agent_one,B1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_blue.append(e.availibility-1)
    env_availability_score_blue.append(np.sum(e.value[e.availibility == item]))  

In [23]:
top_five_blue = sorted( [(x,i) for (i,x) in enumerate(env_availability_score_blue)], reverse=True )[:15] 
top_five_blue_origins = []
for item in top_five_blue:
    top_five_blue_origins.append(All_coordinates[item[1]])
top_five_blue_origins

[574, 497, 657, 503, 580, 651]

# Green function max simulation

In [24]:
env_availability_viz_green= []
env_availability_score_green= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_G = {"availibility": occ_lattice_sim,"enviornment": Gv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_G, Agent_one,G1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_blue.append(e.availibility-1)
    env_availability_score_green.append(np.sum(e.value[e.availibility == item]))  

In [25]:
top_five_green = sorted( [(x,i) for (i,x) in enumerate(env_availability_score_green)], reverse=True )[:15] 
top_five_green_origins = []
for item in top_five_green:
    top_five_green_origins.append(All_coordinates[item[1]])
top_five_green_origins    

[574, 580, 497, 651, 503, 657]

# Yellow function max simulation

In [26]:
env_availability_viz_yellow= []
env_availability_score_yellow= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_Y = {"availibility": occ_lattice_sim,"enviornment": Yv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_Y, Agent_one,Y1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_yellow.append(e.availibility-1)
    env_availability_score_yellow.append(np.sum(e.value[e.availibility == item]))  

In [27]:
top_five_yellow = sorted( [(x,i) for (i,x) in enumerate(env_availability_score_yellow)], reverse=True )[:15] 
top_five_yellow_origins = []
for item in top_five_yellow:
    top_five_yellow_origins.append(All_coordinates[item[1]])
top_five_yellow_origins    

[580, 651, 503, 497, 574, 657]

# Red function max simulation

In [28]:
env_availability_viz_red= []
env_availability_score_red= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_R = {"availibility": occ_lattice_sim,"enviornment": Rv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_R, Agent_one,R1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_red.append(e.availibility-1)
    env_availability_score_red.append(np.sum(e.value[e.availibility == item]))  

In [29]:
top_five_red = sorted( [(x,i) for (i,x) in enumerate(env_availability_score_red)], reverse=True )[:15] 
top_five_red_origins = []
for item in top_five_red:
    top_five_red_origins.append(All_coordinates[item[1]])
top_five_red_origins  

[651, 657, 497, 503, 580, 574]

# Violet function max simulation

In [30]:
env_availability_viz_violet= []
env_availability_score_violet= []
for item in All_coordinates:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item)
    
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
    env_V = {"availibility": occ_lattice_sim,"enviornment": Vv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_V, Agent_one,V1,"one_neighbour_occupy_cubish_behaviour")
   
    env_list =[env_1]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_violet.append(e.availibility-1)
    env_availability_score_violet.append(np.sum(e.value[e.availibility == item]))  

In [31]:
top_five_violet = sorted( [(x,i) for (i,x) in enumerate(env_availability_score_violet)], reverse=True )[:15] 
top_five_violet_origins = []
for item in top_five_violet:
    top_five_violet_origins.append(All_coordinates[item[1]])
top_five_violet

[(17233, 0), (17175, 1), (16883, 3), (16869, 2), (16820, 5), (16711, 4)]

# All_simulations

In [32]:
env_availability_viz_all_options = []

for item in all_permutations:
    Agent_one=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[0])
    Agent_two=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[1])
    Agent_three=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[2])
    Agent_four=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[3])
    Agent_five=initialize_agents_fixed_origin (stencil_von_neumann,avail_lattice,item[4])
    # name the lattices myagent_attractor_one
    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
    env_B = {"availibility": occ_lattice_sim,"enviornment": Bv}
    env_G = {"availibility": occ_lattice_sim,"enviornment": Gv}
    env_R = {"availibility": occ_lattice_sim,"enviornment": Rv}
    env_Y = {"availibility": occ_lattice_sim,"enviornment": Yv}
    env_V = {"availibility": occ_lattice_sim,"enviornment": Vv}


    # initiate the environment
    # Replace the envs in the enviornment initilization to check the values for all the agents on all the locations
    env_1 = environment(env_B, Agent_one,B1,"one_neighbour_occupy_cubish_behaviour")
    env_2 = environment(env_G, Agent_two,G1,"one_neighbour_occupy_cubish_behaviour")
    env_3 = environment(env_Y, Agent_three,Y1,"one_neighbour_occupy_cubish_behaviour")
    env_4 = environment(env_R, Agent_four,R1,"one_neighbour_occupy_cubish_behaviour")
    env_5 = environment(env_V, Agent_five,V1,"one_neighbour_occupy_cubish_behaviour")
    
    
    env_list =[env_2,env_3,env_4,env_5]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  
                    
    env_availability_viz_all_options.append(e.availibility)    


## Pickle Objects for reference 
change the name for each simulation / put it in a for loop 

In [33]:
all_options = pickle.dump( env_availability_viz_all_options, open( "all_options.p", "wb" ) )
pickle_all_options = pickle.load( open( "all_options.p", "rb" ) ) 
pickle_all_options_array = np.array(pickle_all_options)
pickle_all_options_tglattice=tg.to_lattice(np.copy(pickle_all_options_array), pickle_all_options_array.shape)
pickle_all_options_tglattice.shape

(720, 6, 21, 11)

## Vizualise the simulation

In [34]:
p = pv.Plotter(notebook=True)

base_lattice = pickle_all_options_tglattice[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
#base_env_availability_viz_top_500[0].fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = pickle_all_options_tglattice[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.0, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = len(all_permutations)
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(749.7749358870385, 43.27493588703855, 53.27493588703855),
 (722.5, 16.0, 26.0),
 (0.0, 0.0, 1.0)]

# Calculate the score for each simulation

In [35]:
all_values=[]
for lattice,index in zip (pickle_all_options,all_permutations):
    lattice_flat = lattice.flatten()
    B= np.sum(Bv.flatten()[lattice_flat == index[0]])
    G= np.sum(Gv.flatten()[lattice_flat == index[1]])
    Y= np.sum(Yv.flatten()[lattice_flat == index[2]])
    R= np.sum(Rv.flatten()[lattice_flat == index[3]])
    V= np.sum(Vv.flatten()[lattice_flat == index[4]])
    all_values.append(B+G+Y+R+V)
sorted_values= np.sort(np.array(all_values))

In [45]:
np.argwhere(sorted_values==17131)

array([[377]], dtype=int64)

In [47]:
sorted_values

array([ 8450,  8452,  9325,  9327,  9382,  9384, 11508, 11537, 11542,
       11569, 12180, 12313, 13023, 13109, 13113, 13211, 13409, 13602,
       13689, 13692, 13739, 13746, 13817, 13840, 13847, 13850, 13873,
       13882, 13927, 13964, 13996, 14008, 14062, 14073, 14096, 14188,
       14192, 14224, 14264, 14353, 14467, 14504, 14520, 14550, 14623,
       14723, 14738, 14771, 14783, 14888, 14904, 14916, 14925, 14976,
       14981, 15037, 15104, 15116, 15127, 15146, 15203, 15218, 15223,
       15223, 15234, 15235, 15244, 15276, 15329, 15340, 15342, 15345,
       15347, 15352, 15354, 15355, 15362, 15378, 15474, 15481, 15483,
       15496, 15500, 15500, 15539, 15546, 15565, 15576, 15577, 15581,
       15584, 15595, 15597, 15604, 15608, 15625, 15641, 15678, 15680,
       15734, 15753, 15765, 15774, 15781, 15789, 15795, 15797, 15810,
       15819, 15821, 15825, 15832, 15839, 15843, 15870, 15894, 15897,
       15897, 15899, 15912, 15913, 15913, 15920, 15950, 15955, 15989,
       15992, 16021,

In [37]:
sorted_values[-1]

18162

In [48]:
np.argwhere(sorted_values==16546)

array([[204]], dtype=int64)

In [39]:
max_value = max(all_values)
max_index = all_values.index(max_value)
print(all_values[max_index])
print(all_values[664])
sorted_val = sorted(all_values)
max_index

18162
14192


496

# Save to Csv

In [40]:
for i, lattice in enumerate(env_availability_viz_all_options):
    csv_path = os.path.relpath('csv/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)


## Linear Optimization to derive a solution

In [41]:
from ortools.linear_solver import pywraplp


def main():
    # Data from all the colours on all the origins
    costs = [
        env_availability_score_blue,
        env_availability_score_green,
        env_availability_score_yellow,
        env_availability_score_red,
        env_availability_score_violet,
        [0,0,0,0,0,0]
    ]
    num_workers = 6

    num_tasks = len(costs[0])
    #print(num_tasks)
    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print('Worker %d assigned to task %d.  Cost = %d' %
                          (i, j, costs[i][j]))


if __name__ == '__main__':
    main()
    


Total cost =  21154.0 

Worker 0 assigned to task 5.  Cost = 1490
Worker 1 assigned to task 2.  Cost = 1538
Worker 2 assigned to task 3.  Cost = 459
Worker 3 assigned to task 4.  Cost = 434
Worker 4 assigned to task 0.  Cost = 17233
Worker 5 assigned to task 1.  Cost = 0


## Total cost of permutation matrix

In [42]:
cost_for_location_filter_1 = [env_availability_viz[-1] == 193]
cost_for_location_filter_2 = [env_availability_viz[-1] == 189]
cost_for_location_filter_3 = [env_availability_viz[-1] == 256]
cost_for_location_filter_4 = [env_availability_viz[-1] == 252]
cost_for_location_filter_5 = [env_availability_viz[-1] == 306]
cost_for_location_filter_6 = [env_availability_viz[-1] == 310]

cost = [(np.sum(Rv[cost_for_location_filter_2]))+(np.sum(Yv[cost_for_location_filter_3]))+(np.sum(Vv[cost_for_location_filter_4]))+(np.sum(Bv[cost_for_location_filter_5]))+(np.sum(Gv[cost_for_location_filter_6]))]
cost

NameError: name 'env_availability_viz' is not defined

In [ ]:
avail_lattice

In [ ]:
for i, lattice in enumerate(env_availability_viz):
    csv_path = os.path.relpath('csv/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)